In [1]:
from numba import jit, cuda

from imageai.Detection import ObjectDetection
import os
import sys
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

2022-11-04 13:21:44.196926: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/natnael_masresha/week11/.imageai/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-04 13:21:44.196963: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#!/home/natnael_masresha/week11/.venv/bin/python

sys.path.append("../scripts")
from load_data import Loader

loader = Loader()

#!/home/natnael_masresha/week11/.imageai/bin/python


In [3]:
# constants

IMAGES_EXT = ["JPG","PNG","GIF","WEBP","TIFF","PSD","RAW","BMP","HEIF","INDD","JPEG"]

VIDEO_EXT = ["WEBM","MPG","MP2","MPEG","MPE","MPV","OGG","MP4","M4P","M4V","AVI","WMV","MOV","QT","FLV","SWF"]

In [4]:
def get_files_name(directory:str, filter_extension:list=None,exclude_names:list=None)->list:
    
    # directory = f'/home/amanuel_zewdu/creative_image_optimization/data/Challenge_Data/Assets/{directory}/'
    
    files = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        
        # checking if it is a file
        if filter_extension != None:
            if os.path.isfile(f):
                if ~any(substring in filename.split('.')[0] for substring in exclude_names):
                    if filename.split('.')[-1].upper() in filter_extension:
                        files.append(filename)
        else:
            if os.path.isfile(f):
                if ~any(substring in filename.split('.')[0] for substring in exclude_names):
                    files.append(filename)

    return files

In [5]:
def get_objects(img_path:str,model_path:str = None,print_objects:bool = False):
    
    detector = ObjectDetection()
    execution_path = os.getcwd()
    if model_path == None:
        model_path = "../junk/data/resnet50_coco_best_v2.1.0.h5"
    
    detector.setModelTypeAsRetinaNet()
    detector.setModelPath( os.path.join(execution_path , model_path))
    detector.loadModel()
    detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , img_path), output_image_path=os.path.join(execution_path , model_path+"newI.jpg"))

    objects = []

    for eachObject in detections:
        objects.append(eachObject["name"])
        if print_objects:
            print(eachObject["name"] , " : " , eachObject["percentage_probability"] )
    return objects

In [6]:
def get_unique_objects(all_objects:list):
    unique_objects = list(set(all_objects))
    unique_objects_count = len(unique_objects)
    all_objects_count = len(all_objects)
    
    return all_objects,all_objects_count,unique_objects_count

In [7]:
# all in one
@jit(target_backend='cuda') 
def add_feature(game_id:str):
    path  = f'/home/amanuel_zewdu/creative_image_optimization/data/Challenge_Data/Assets/{game_id}/'
    exclude = ["text","video","background"]
    file_names = get_files_name(path,IMAGES_EXT,exclude)
    all_objects = []
    for file_name in file_names:
        all_text = get_objects(path+file_name)
        all_objects.extend(all_text)
    return get_unique_objects(all_objects)

In [8]:
bucket = "s3://10ac-batch-6/data/w11/Challenge_Data.zip"
file_path = "Challenge_Data/performance_data.csv"

df = loader.load_csv(bucket,file_path)

In [9]:
tst_df = df.sample(1)

In [10]:
tst_df[['all_objects','all_objects_count','unique_objects_count']] = tst_df.game_id.apply(lambda x:pd.Series(add_feature(x)))

2022-11-04 13:21:53.974163: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-04 13:21:53.974256: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-04 13:21:56.452957: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 13:21:56.453648: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: NVIDIA A10G computeCapability: 8.6
coreClock: 1.71GHz coreCount: 80 deviceMemorySize: 22.06GiB deviceMemoryBandwidth: 558.88GiB/s
2022-11-04 13:21:56.453729: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or direct

2022-11-04 13:22:01.315411: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-11-04 13:22:01.411200: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2799910000 Hz


In [11]:
tst_df

,game_id,preview_link,ER,CTR,all_objects,all_objects_count,unique_objects_count
59,adunit-van-cleef-and-arpels-frivole-wave-unit1...,https://s3.ap-southeast-1.amazonaws.com/a.futu...,0.1135966328541683,0.0142691712411896,"[sports ball, sports ball, sports ball]",3,1
